In [87]:
import pandas as pd
from sklearn.model_selection import train_test_split
!pip install pgmpy
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.estimators import BayesianEstimator

In [0]:
acs = pd.read_csv('acs_nyc_tracts_5yr_2009_2017.csv')

In [128]:
acs.shape

(19554, 341)

In [91]:
acs.head()

,year,span,sum_level,geoid_full,geoid,geo_name,geo_type,pop_num,pop_u18_num,pop_65p_num,...,hh_gini_idx,cpi_new,hh_inc_med_adj,hh_inc_own_med_adj,hh_inc_rent_med_adj,own_cost_mort_med_adj,own_cost_nomort_med_adj,rent_gross_med_adj,rent_contract_med_adj,cpi_year
0,2009,5,140,14000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",tract,11355,471,33,...,NaN,0.865459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
1,2010,5,140,14000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",tract,11517,484,44,...,NaN,0.880219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
2,2011,5,140,14000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",tract,10453,631,60,...,NaN,0.905266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
3,2012,5,140,14000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",tract,9741,595,56,...,NaN,0.923063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
4,2013,5,140,14000US36005000100,36005000100,"Census Tract 1, Bronx County, New York",tract,9191,480,42,...,NaN,0.938576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018


In [0]:
crime = pd.read_csv('crime_nyc_tract.csv')

In [95]:
crime.head()

,year,fc_geoid,fc_geo_type,census_denom_year,crime_prop_num,crime_viol_num,crime_all_num,population,crime_prop_rt,crime_viol_rt,crime_all_rt
0,2006,36005000100,tract,2000,1.0,4.0,5.0,12780,0.078247,0.312989,0.391236
1,2006,36005000200,tract,2000,34.0,23.0,57.0,3545,9.590973,6.488011,16.078984
2,2006,36005000400,tract,2000,38.0,12.0,50.0,3314,11.466506,3.621002,15.087508
3,2006,36005001100,tract,2000,21.0,25.0,46.0,559,37.567084,44.722719,82.289803
4,2006,36005001500,tract,2000,2.0,2.0,4.0,19,105.263158,105.263158,210.526316


In [0]:
vio = pd.read_csv('hpd_vio_count.csv')

In [99]:
vio.head()

,Unnamed: 0,year,fc_geoid,fc_geo_type,violation_class,viol
0,1,2012,101,cd,A,23
1,2,2012,101,cd,B,129
2,3,2012,101,cd,C,48
3,4,2012,101,sba,A,281
4,5,2012,101,sba,B,1023


HPD Violation Count by year, tract, class

In [0]:
def vio_by_class(df):
  output = df[(df['fc_geo_type']=='tract') & (df['year']>=2014)][['year', 'fc_geoid', 'violation_class', 
                                                 'viol']].groupby(['year','fc_geoid','violation_class']).sum().unstack(level=-1)
  output.columns = output.columns.to_flat_index()
  output.columns = ['class_A_count', 'class_B_count', 'class_C_count']
  output.reset_index(inplace=True)
  output.fillna(0, inplace=True)
  output['total_vio_count'] = output['class_A_count'] + output['class_B_count'] + output['class_C_count']
  output = output.drop(columns=['class_A_count', 'class_B_count'])
  return output

In [101]:
vio_by_class = vio_by_class(vio)
vio_by_class.head()

,year,fc_geoid,class_C_count,total_vio_count
0,2014,36005000200,63.0,222.0
1,2014,36005000400,57.0,191.0
2,2014,36005001600,9.0,68.0
3,2014,36005001900,55.0,192.0
4,2014,36005002000,41.0,220.0


Crime Rate

In [0]:
def crime_cleaned(df):
  output = df[df['year']>=2014][['year', 'fc_geoid', 'crime_prop_rt', 'crime_viol_rt']]
  return output

In [103]:
crime_cleaned = crime_cleaned(crime)
crime_cleaned.head()

,year,fc_geoid,crime_prop_rt,crime_viol_rt
17446,2014,36005000200,7.977026,5.105297
17447,2014,36005000400,5.789076,5.033979
17448,2014,36005001600,9.706382,8.493084
17449,2014,36005001900,47.419805,22.315202
17450,2014,36005002000,8.288640,13.164310


ACS

In [0]:
def acs_cleaned1(df):
  output = df[df.year>=2014][['year', 'geoid', 'pop_num', 'pop_nonfam_pct', 'pop_race_div_idx', 'pop_pov_pct', 'pop_edu_collp_pct',
                              'unit_vac_pct', 'unit_occ_rent_sevcrowd_pct', 'rent_gross_med', 'hh_inc_med', 'hh_gini_idx', 'unit_num']]
  return output

In [0]:
def acs_cleaned2(df):
  output = df[df.year>=2014][['year', 'geoid', 'hh_family_pct', 'pop_race_div_idx', 'pop_pov_pct', 'pop_edu_collp_pct',
                              'unit_occ_rent_sevcrowd_pct', 'rent_gross_med', 'hh_gini_idx', 'unit_num',
                              'pop_school_none_5_17_pct', 'rent_burden_sev_pct','own_burden_pct', 'pop_num',
                              'pop_mobile1_abroad_pct','pop_commute_carfree_pct', 'pop_5p_units_pct', 'pop_65p_pct', 'pop_disabled_pct']]
  return output

In [0]:
def acs_cleaned3(df):
  output = df[df.year>=2014][['year', 'geoid', 'hh_alone_pct', 'pop_pov_pct', 'pop_edu_nohs_pct',
                              'unit_occ_rent_sevcrowd_pct', 'rent_gross_med', 'hh_gini_idx', 'unit_num',
                              'rent_burden_sev_pct','own_burden_pct', 'pop_num', 
                              'pop_foreign_pct', 'pop_mobile1_abroad_pct','pop_5p_units_pct', 'pop_65p_pct', 'pop_disabled_pct', 
                              'pop_school_priv_5_17_pct', 'pop_lim_eng_pct', 'unit_occ_rent_nochild_pct', 'unit_occ_hhsize_avg', 
                              'pop_rent_pct', 'hh_u18_pct','pop_commute_time_avg','pop_commute_pubtrans_pct']]
  return output

In [0]:
def acs_cleaned4(df):
  output = df[df.year>=2014][['year', 'geoid', 'hh_inc_med', 'hh_alone_pct', 'pop_pov_pct', 'pop_edu_nohs_pct','pop_num', 
                              'rent_gross_med', 'unit_num', 'rent_burden_sev_pct','own_burden_pct', 'pop_65p_pct', 'pop_disabled_pct', 
                              'pop_school_priv_5_17_pct', 'pop_lim_eng_pct', 'unit_occ_hhsize_avg', 'pop_rent_pct']]
  return output

In [0]:
def acs_cleaned5(df):
  output = df[df.year>=2014][['year', 'geoid', 'pop_pov_pct', 'unit_num', 'pop_65p_pct', 'pop_lim_eng_pct', 'pop_rent_pct', 'pop_num']]
  return output

In [0]:
def acs_cleaned51(df):
  output = df[df.year>=2014][['year', 'geoid', 'pop_pov_pct', 'unit_num', 'pop_65p_pct', 'pop_lim_eng_pct', 'pop_num']]
  return output

In [0]:
def acs_cleaned52(df):
  output = df[df.year>=2014][['year', 'geoid', 'unit_num', 'pop_65p_pct', 'pop_lim_eng_pct', 'pop_rent_pct', 'pop_num']]
  return output

In [0]:
def acs_cleaned6(df):
  output = df[df.year>=2014][['year', 'geoid', 'hh_alone_pct', 'pop_pov_pct', 'pop_edu_nohs_pct', 'unit_vac_pct',
                              'unit_occ_rent_sevcrowd_pct', 'rent_gross_med', 'hh_gini_idx', 'unit_num', 'hh_inc_med',
                              'rent_burden_sev_pct','own_burden_pct', 'pop_num', 'rent_burden_sev_pct', 
                              'pop_foreign_pct', 'pop_mobile1_abroad_pct','pop_5p_units_pct', 'pop_65p_pct', 'pop_disabled_pct', 
                              'pop_school_priv_5_17_pct', 'pop_lim_eng_pct', 'unit_occ_rent_nochild_pct', 'unit_occ_hhsize_avg', 
                              'pop_rent_pct', 'hh_u18_pct','pop_commute_time_avg','pop_commute_pubtrans_pct']]
  return output

In [0]:
acs_cleaned1 = acs_cleaned1(acs)
acs_cleaned2 = acs_cleaned2(acs)
acs_cleaned3 = acs_cleaned3(acs)
acs_cleaned4 = acs_cleaned4(acs)
acs_cleaned5 = acs_cleaned5(acs)
acs_cleaned51 = acs_cleaned51(acs)
acs_cleaned52 = acs_cleaned52(acs)
acs_cleaned6 = acs_cleaned6(acs)


Join three datasets

In [0]:
df1 = vio_by_class.merge(crime_cleaned, on=['fc_geoid','year']) \
.merge(acs_cleaned1, left_on=['year', 'fc_geoid'], right_on=['year', 'geoid']).drop(columns=['geoid']).dropna()
df2 = vio_by_class.merge(crime_cleaned, on=['fc_geoid','year']) \
.merge(acs_cleaned2, left_on=['year', 'fc_geoid'], right_on=['year', 'geoid']).drop(columns=['geoid']).dropna()
df3 = vio_by_class.merge(crime_cleaned, on=['fc_geoid','year']) \
.merge(acs_cleaned3, left_on=['year', 'fc_geoid'], right_on=['year', 'geoid']).drop(columns=['geoid']).dropna()
df4 = vio_by_class.merge(crime_cleaned, on=['fc_geoid','year']) \
.merge(acs_cleaned4, left_on=['year', 'fc_geoid'], right_on=['year', 'geoid']).drop(columns=['geoid']).dropna()
df5 = vio_by_class.merge(crime_cleaned, on=['fc_geoid','year']) \
.merge(acs_cleaned5, left_on=['year', 'fc_geoid'], right_on=['year', 'geoid']).drop(columns=['geoid']).dropna()
df51 = vio_by_class.merge(crime_cleaned, on=['fc_geoid','year']) \
.merge(acs_cleaned51, left_on=['year', 'fc_geoid'], right_on=['year', 'geoid']).drop(columns=['geoid']).dropna()

In [0]:
df52 = vio_by_class.merge(crime_cleaned, on=['fc_geoid','year']) \
.merge(acs_cleaned52, left_on=['year', 'fc_geoid'], right_on=['year', 'geoid']).drop(columns=['geoid']).dropna()
df6 = vio_by_class.merge(crime_cleaned, on=['fc_geoid','year']) \
.merge(acs_cleaned6, left_on=['year', 'fc_geoid'], right_on=['year', 'geoid']).drop(columns=['geoid']).dropna()

Bayesian Network

In [0]:
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.estimators import BayesianEstimator

def df_total_(df):
  df['total_vio_count'] = df['unit_num']/df['total_vio_count']
  df=df.drop(columns=['unit_num'])
  df_total = df.drop(columns=['class_C_count'])
  return df_total

def df_classC_(df):
  df['class_C_count'] = df['unit_num']/df['class_C_count']
  df=df.drop(columns=['unit_num'])
  df_classC = df.drop(columns=['total_vio_count'])
  return df_classC

def qcut(data):
  for i in range(data.shape[1]):
    data.iloc[:,i] = pd.qcut(data.iloc[:,i].rank(method='first'), 5, labels=False)
  return data


#df - df1,2,3,4,5,51
#year - 2014, 2015, 2016, 2017
#count - 'Total', 'Class C'
def BN(df,year,count):
  print('For {}:'.format(year))
  print
  df_total = df_total_(df)
  df_classC = df_classC_(df)
  
  if count=='Total':
    data_ = qcut(df_total[df_total['year']==year].iloc[:,2:])
  elif count=='Class C':
    data_ = qcut(df_classC[df_classC['year']==year].iloc[:,2:])
  
  print('{}:'.format(count))
  train,test=train_test_split(data_,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print
  print('Local Independencies:')
  print(best_model.local_independencies(best_model.nodes()))
  print
  print('Active Trail Nodes:')
  for thenode in best_model.nodes():
    print best_model.active_trail_nodes(thenode)
  print
  print('CPD:')

  
  if count=='Total':
    print best_model.get_cpds('total_vio_count')
    print
    print "In sample:",(train.loc[:,'total_vio_count']==best_model.predict(train.drop(columns=['total_vio_count'])) \
                            ['total_vio_count']).mean()
    print "Out of sample:",(test.loc[:,'total_vio_count']==best_model.predict(test.drop(columns=['total_vio_count'])) \
                            ['total_vio_count']).mean()
  elif count=='Class C': 
    print best_model.get_cpds('class_C_count')
    print
    print "In sample:",(train.loc[:,'class_C_count']==best_model.predict(train.drop(columns=['class_C_count'])) \
                            ['class_C_count']).mean()
    print "Out of sample:",(test.loc[:,'class_C_count']==best_model.predict(test.drop(columns=['class_C_count'])) \
                            ['class_C_count']).mean()

Best Models

In [127]:
BN(df4,2014,'Total')

For 2014:

Total:
Best Model:
[('pop_pov_pct', 'hh_inc_med'), ('pop_pov_pct', 'pop_65p_pct'), ('crime_viol_rt', 'crime_prop_rt'), ('hh_inc_med', 'crime_viol_rt'), ('hh_inc_med', 'pop_school_priv_5_17_pct'), ('hh_inc_med', 'rent_burden_sev_pct'), ('hh_inc_med', 'pop_rent_pct'), ('hh_inc_med', 'rent_gross_med'), ('hh_inc_med', 'pop_edu_nohs_pct'), ('rent_gross_med', 'pop_disabled_pct'), ('rent_gross_med', 'hh_alone_pct'), ('pop_rent_pct', 'total_vio_count'), ('hh_alone_pct', 'unit_occ_hhsize_avg'), ('pop_edu_nohs_pct', 'pop_lim_eng_pct'), ('unit_occ_hhsize_avg', 'own_burden_pct'), ('total_vio_count', 'pop_num')]

Local Independencies:
(crime_viol_rt _|_ pop_pov_pct, pop_school_priv_5_17_pct, rent_burden_sev_pct, own_burden_pct, pop_lim_eng_pct, rent_gross_med, pop_65p_pct, pop_rent_pct, hh_alone_pct, pop_edu_nohs_pct, unit_occ_hhsize_avg, pop_disabled_pct, total_vio_count, pop_num | hh_inc_med)
(hh_inc_med _|_ pop_65p_pct | pop_pov_pct)
(pop_school_priv_5_17_pct _|_ pop_pov_pct, crime_vi

In [0]:
BN(df3,2014,'Class C')

For 2014:

Class C:
Best Model:
[('unit_occ_hhsize_avg', 'hh_u18_pct'), ('pop_rent_pct', 'unit_occ_rent_sevcrowd_pct'), ('pop_rent_pct', 'pop_5p_units_pct'), ('pop_rent_pct', 'pop_commute_pubtrans_pct'), ('pop_rent_pct', 'class_C_count'), ('class_C_count', 'crime_viol_rt'), ('pop_5p_units_pct', 'hh_alone_pct'), ('pop_5p_units_pct', 'own_burden_pct'), ('pop_edu_nohs_pct', 'pop_pov_pct'), ('crime_viol_rt', 'crime_prop_rt'), ('crime_viol_rt', 'pop_school_priv_5_17_pct'), ('pop_lim_eng_pct', 'pop_edu_nohs_pct'), ('pop_foreign_pct', 'pop_lim_eng_pct'), ('pop_foreign_pct', 'unit_occ_rent_nochild_pct'), ('rent_gross_med', 'pop_disabled_pct'), ('hh_alone_pct', 'unit_occ_hhsize_avg'), ('hh_u18_pct', 'rent_burden_sev_pct'), ('hh_u18_pct', 'pop_commute_time_avg'), ('hh_u18_pct', 'pop_65p_pct'), ('pop_pov_pct', 'hh_gini_idx'), ('pop_pov_pct', 'pop_rent_pct'), ('pop_pov_pct', 'rent_gross_med')]

Local Independencies:
(pop_school_priv_5_17_pct _|_ unit_occ_rent_sevcrowd_pct, pop_mobile1_abroad_pct, 

In [0]:
BN(df3,2015,'Total')
BN(df51,2015,'Class C')

For 2015:

Total:
Best Model:
[('unit_occ_hhsize_avg', 'hh_u18_pct'), ('unit_occ_hhsize_avg', 'rent_burden_sev_pct'), ('pop_rent_pct', 'unit_occ_rent_sevcrowd_pct'), ('pop_rent_pct', 'pop_commute_pubtrans_pct'), ('pop_rent_pct', 'total_vio_count'), ('pop_5p_units_pct', 'hh_alone_pct'), ('pop_5p_units_pct', 'own_burden_pct'), ('pop_5p_units_pct', 'pop_rent_pct'), ('pop_edu_nohs_pct', 'pop_pov_pct'), ('pop_edu_nohs_pct', 'pop_school_priv_5_17_pct'), ('crime_viol_rt', 'crime_prop_rt'), ('pop_lim_eng_pct', 'pop_edu_nohs_pct'), ('pop_lim_eng_pct', 'pop_foreign_pct'), ('hh_gini_idx', 'pop_5p_units_pct'), ('rent_gross_med', 'pop_disabled_pct'), ('hh_alone_pct', 'unit_occ_hhsize_avg'), ('hh_alone_pct', 'unit_occ_rent_nochild_pct'), ('hh_u18_pct', 'pop_commute_time_avg'), ('hh_u18_pct', 'pop_65p_pct'), ('pop_pov_pct', 'crime_viol_rt'), ('pop_pov_pct', 'hh_gini_idx'), ('pop_pov_pct', 'rent_gross_med')]

Local Independencies:
(pop_school_priv_5_17_pct _|_ crime_viol_rt, pop_mobile1_abroad_pct, re

In [0]:
BN(df3,2016,'Total')
BN(df1,2016,'Class C')

For 2016:

Total:
Best Model:
[('unit_occ_hhsize_avg', 'hh_alone_pct'), ('unit_occ_hhsize_avg', 'hh_u18_pct'), ('unit_occ_hhsize_avg', 'rent_burden_sev_pct'), ('unit_occ_hhsize_avg', 'pop_foreign_pct'), ('pop_rent_pct', 'pop_pov_pct'), ('pop_rent_pct', 'unit_occ_rent_sevcrowd_pct'), ('pop_rent_pct', 'total_vio_count'), ('pop_rent_pct', 'pop_commute_pubtrans_pct'), ('pop_5p_units_pct', 'own_burden_pct'), ('pop_5p_units_pct', 'hh_gini_idx'), ('pop_5p_units_pct', 'pop_rent_pct'), ('pop_edu_nohs_pct', 'pop_school_priv_5_17_pct'), ('crime_viol_rt', 'crime_prop_rt'), ('pop_foreign_pct', 'pop_lim_eng_pct'), ('total_vio_count', 'crime_viol_rt'), ('rent_gross_med', 'pop_disabled_pct'), ('hh_alone_pct', 'pop_5p_units_pct'), ('hh_alone_pct', 'unit_occ_rent_nochild_pct'), ('hh_u18_pct', 'pop_commute_time_avg'), ('hh_u18_pct', 'pop_65p_pct'), ('pop_pov_pct', 'pop_edu_nohs_pct'), ('pop_pov_pct', 'rent_gross_med')]

Local Independencies:
(pop_school_priv_5_17_pct _|_ crime_viol_rt, pop_mobile1_abroad

In [0]:
BN(df5,2017,'Total')
BN(df2,2017,'Class C')

For 2017:

Total:
Best Model:
[('pop_pov_pct', 'pop_lim_eng_pct'), ('pop_pov_pct', 'pop_rent_pct'), ('crime_viol_rt', 'crime_prop_rt'), ('pop_rent_pct', 'pop_65p_pct'), ('pop_rent_pct', 'total_vio_count'), ('total_vio_count', 'crime_viol_rt')]

Local Independencies:
(crime_viol_rt _|_ pop_pov_pct, pop_lim_eng_pct, pop_rent_pct, pop_65p_pct | total_vio_count)
(pop_lim_eng_pct _|_ crime_viol_rt, crime_prop_rt, pop_65p_pct, pop_rent_pct, total_vio_count | pop_pov_pct)
(pop_rent_pct _|_ pop_lim_eng_pct | pop_pov_pct)
(pop_65p_pct _|_ pop_pov_pct, crime_viol_rt, crime_prop_rt, pop_lim_eng_pct, total_vio_count | pop_rent_pct)
(crime_prop_rt _|_ pop_pov_pct, pop_65p_pct, pop_lim_eng_pct, pop_rent_pct, total_vio_count | crime_viol_rt)
(total_vio_count _|_ pop_pov_pct, pop_lim_eng_pct, pop_65p_pct | pop_rent_pct)

Active Trail Nodes:
{'pop_pov_pct': set(['pop_pov_pct', 'crime_viol_rt', 'pop_lim_eng_pct', 'pop_rent_pct', 'pop_65p_pct', 'crime_prop_rt', 'total_vio_count'])}
{'crime_viol_rt': set(

## **Below are working process of fitting different set of features (Don't RUN)** 

Feature set 5

In [0]:
df['total_vio_count'] = df['unit_num']/df['total_vio_count']
df['class_C_count'] = df['unit_num']/df['class_C_count']
df=df.drop(columns=['unit_num','pop_pov_pct'])
df_classC = df.drop(columns=['total_vio_count'])
df_total = df.drop(columns=['class_C_count'])

In [0]:
df.columns

Index([u'year', u'fc_geoid', u'class_C_count', u'total_vio_count',
       u'crime_prop_rt', u'crime_viol_rt', u'pop_65p_pct', u'pop_lim_eng_pct',
       u'pop_rent_pct'],
      dtype='object')

In [0]:
def qcut(data):
  for i in range(data.shape[1]):
    data.iloc[:,i] = pd.qcut(data.iloc[:,i].rank(method='first'), 5, labels=False)
  return data

for i in [2014,2015,2016,2017]:
  print('For {}:'.format(i))
  print
  total = qcut(df_total[df_total['year']==i].iloc[:,2:])
  classC = qcut(df_classC[df_classC['year']==i].iloc[:,2:])
  
  print('Total Count:')
  train,test=train_test_split(total,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'total_vio_count']==best_model.predict(train.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print "Out of sample:",(test.loc[:,'total_vio_count']==best_model.predict(test.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print 
  print('Class C:')
  train,test=train_test_split(classC,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'class_C_count']==best_model.predict(train.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print "Out of sample:",(test.loc[:,'class_C_count']==best_model.predict(test.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print
  print
  

For 2014:

Total Count:
Best Model:
[('crime_viol_rt', 'total_vio_count'), ('pop_rent_pct', 'pop_lim_eng_pct'), ('crime_prop_rt', 'crime_viol_rt'), ('total_vio_count', 'pop_rent_pct'), ('total_vio_count', 'pop_65p_pct')]
In sample: 0.4387317909168809
Out of sample: 0.44087403598971725

Class C:
Best Model:
[('crime_viol_rt', 'class_C_count'), ('pop_rent_pct', 'pop_lim_eng_pct'), ('class_C_count', 'pop_rent_pct'), ('class_C_count', 'pop_65p_pct'), ('crime_prop_rt', 'crime_viol_rt')]
In sample: 0.4464438731790917
Out of sample: 0.42930591259640105


For 2015:

Total Count:
Best Model:
[('crime_viol_rt', 'crime_prop_rt'), ('pop_rent_pct', 'pop_lim_eng_pct'), ('pop_rent_pct', 'total_vio_count'), ('pop_65p_pct', 'pop_rent_pct'), ('total_vio_count', 'crime_viol_rt')]
In sample: 0.4447278911564626
Out of sample: 0.4017857142857143

Class C:
Best Model:
[('crime_viol_rt', 'pop_65p_pct'), ('pop_rent_pct', 'pop_lim_eng_pct'), ('pop_rent_pct', 'class_C_count'), ('pop_65p_pct', 'pop_rent_pct'), ('

Feature set 4

In [0]:
df['total_vio_count'] = df['unit_num']/df['total_vio_count']
df['class_C_count'] = df['unit_num']/df['class_C_count']
df=df.drop(columns=['unit_num'])
df_classC = df.drop(columns=['total_vio_count'])
df_total = df.drop(columns=['class_C_count'])

In [0]:
for i in [2014,2015,2016,2017]:
  print i
  print df[df['year']==i].shape
  

2014
(1819, 19)
2015
(1770, 19)
2016
(1951, 19)
2017
(1940, 19)


In [0]:
print(df_total.columns)
df_total.head()

Index([u'year', u'fc_geoid', u'total_vio_count', u'crime_prop_rt',
       u'crime_viol_rt', u'hh_inc_med', u'hh_alone_pct', u'pop_pov_pct',
       u'pop_edu_nohs_pct', u'rent_gross_med', u'rent_burden_sev_pct',
       u'own_burden_pct', u'pop_65p_pct', u'pop_disabled_pct',
       u'pop_school_priv_5_17_pct', u'pop_lim_eng_pct', u'unit_occ_hhsize_avg',
       u'pop_rent_pct'],
      dtype='object')


,year,fc_geoid,total_vio_count,crime_prop_rt,crime_viol_rt,hh_inc_med,hh_alone_pct,pop_pov_pct,pop_edu_nohs_pct,rent_gross_med,rent_burden_sev_pct,own_burden_pct,pop_65p_pct,pop_disabled_pct,pop_school_priv_5_17_pct,pop_lim_eng_pct,unit_occ_hhsize_avg,pop_rent_pct
0,2014,36005000200,6.644144,7.977026,5.105297,74837.0,0.145758,0.191997,0.248147,1282.0,0.448669,0.529925,0.147399,0.111076,0.090508,0.278327,3.69108,0.374092
1,2014,36005000400,10.445026,5.789076,5.033979,77991.0,0.151182,0.116187,0.147368,1491.0,0.346512,0.507622,0.092606,0.095979,0.162991,0.137455,3.05607,0.280510
2,2014,36005001600,29.970588,9.706382,8.493084,32354.0,0.303870,0.191159,0.329766,920.0,0.246502,0.592000,0.160200,0.119555,0.085242,0.287729,2.69195,0.689024
3,2014,36005001900,4.661458,47.419805,22.315202,34635.0,0.350120,0.445635,0.264921,1126.0,0.319120,0.370968,0.036979,0.123794,0.032483,0.357023,2.51199,0.756715
4,2014,36005002000,14.404545,8.288640,13.164310,23423.0,0.252349,0.421247,0.314322,601.0,0.283613,0.580321,0.108494,0.214454,0.009434,0.219983,3.00570,0.819288


In [0]:
print(df_classC.columns)
df_classC.head()

Index([u'year', u'fc_geoid', u'class_C_count', u'crime_prop_rt',
       u'crime_viol_rt', u'hh_inc_med', u'hh_alone_pct', u'pop_pov_pct',
       u'pop_edu_nohs_pct', u'rent_gross_med', u'rent_burden_sev_pct',
       u'own_burden_pct', u'pop_65p_pct', u'pop_disabled_pct',
       u'pop_school_priv_5_17_pct', u'pop_lim_eng_pct', u'unit_occ_hhsize_avg',
       u'pop_rent_pct'],
      dtype='object')


,year,fc_geoid,class_C_count,crime_prop_rt,crime_viol_rt,hh_inc_med,hh_alone_pct,pop_pov_pct,pop_edu_nohs_pct,rent_gross_med,rent_burden_sev_pct,own_burden_pct,pop_65p_pct,pop_disabled_pct,pop_school_priv_5_17_pct,pop_lim_eng_pct,unit_occ_hhsize_avg,pop_rent_pct
0,2014,36005000200,23.412698,7.977026,5.105297,74837.0,0.145758,0.191997,0.248147,1282.0,0.448669,0.529925,0.147399,0.111076,0.090508,0.278327,3.69108,0.374092
1,2014,36005000400,35.000000,5.789076,5.033979,77991.0,0.151182,0.116187,0.147368,1491.0,0.346512,0.507622,0.092606,0.095979,0.162991,0.137455,3.05607,0.280510
2,2014,36005001600,226.444444,9.706382,8.493084,32354.0,0.303870,0.191159,0.329766,920.0,0.246502,0.592000,0.160200,0.119555,0.085242,0.287729,2.69195,0.689024
3,2014,36005001900,16.272727,47.419805,22.315202,34635.0,0.350120,0.445635,0.264921,1126.0,0.319120,0.370968,0.036979,0.123794,0.032483,0.357023,2.51199,0.756715
4,2014,36005002000,77.292683,8.288640,13.164310,23423.0,0.252349,0.421247,0.314322,601.0,0.283613,0.580321,0.108494,0.214454,0.009434,0.219983,3.00570,0.819288


In [0]:
def qcut(data):
  for i in range(data.shape[1]):
    data.iloc[:,i] = pd.qcut(data.iloc[:,i].rank(method='first'), 5, labels=False)
  return data

for i in [2014,2015,2016,2017]:
  print('For {}:'.format(i))
  print
  total = qcut(df_total[df_total['year']==i].iloc[:,2:])
  classC = qcut(df_classC[df_classC['year']==i].iloc[:,2:])
  
  print('Total Count:')
  train,test=train_test_split(total,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'total_vio_count']==best_model.predict(train.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print "Out of sample:",(test.loc[:,'total_vio_count']==best_model.predict(test.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print 
  print('Class C:')
  train,test=train_test_split(classC,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'class_C_count']==best_model.predict(train.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print "Out of sample:",(test.loc[:,'class_C_count']==best_model.predict(test.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print
  print
  

For 2014:

Total Count:
Best Model:
[('pop_pov_pct', 'hh_inc_med'), ('crime_viol_rt', 'crime_prop_rt'), ('hh_inc_med', 'pop_edu_nohs_pct'), ('hh_inc_med', 'pop_school_priv_5_17_pct'), ('hh_inc_med', 'rent_burden_sev_pct'), ('hh_inc_med', 'pop_rent_pct'), ('hh_inc_med', 'rent_gross_med'), ('rent_gross_med', 'pop_disabled_pct'), ('rent_gross_med', 'hh_alone_pct'), ('pop_rent_pct', 'total_vio_count'), ('hh_alone_pct', 'unit_occ_hhsize_avg'), ('pop_edu_nohs_pct', 'pop_lim_eng_pct'), ('unit_occ_hhsize_avg', 'own_burden_pct'), ('total_vio_count', 'crime_viol_rt'), ('total_vio_count', 'pop_65p_pct')]
In sample: 0.4307974335472044
Out of sample: 0.4491758241758242

Class C:
Best Model:
[('pop_pov_pct', 'hh_inc_med'), ('crime_viol_rt', 'class_C_count'), ('hh_inc_med', 'pop_edu_nohs_pct'), ('hh_inc_med', 'pop_school_priv_5_17_pct'), ('hh_inc_med', 'rent_burden_sev_pct'), ('hh_inc_med', 'pop_rent_pct'), ('hh_inc_med', 'rent_gross_med'), ('rent_gross_med', 'pop_disabled_pct'), ('rent_gross_med', '

Feature set 3

In [0]:
df['total_vio_count'] = df['unit_num']/df['total_vio_count']
df['class_C_count'] = df['unit_num']/df['class_C_count']
df=df.drop(columns=['unit_num'])
df_classC = df.drop(columns=['total_vio_count'])
df_total = df.drop(columns=['class_C_count'])

In [0]:
for i in [2014,2015,2016,2017]:
  print i
  print df[df['year']==i].shape
  

2014
(1636, 27)
2015
(1777, 27)
2016
(1794, 27)
2017
(1780, 27)


In [0]:
print(df_total.columns)
df_total.head()

Index([u'year', u'fc_geoid', u'total_vio_count', u'crime_prop_rt',
       u'crime_viol_rt', u'hh_alone_pct', u'pop_pov_pct', u'pop_edu_nohs_pct',
       u'unit_occ_rent_sevcrowd_pct', u'rent_gross_med', u'hh_gini_idx',
       u'rent_burden_sev_pct', u'own_burden_pct', u'pop_foreign_pct',
       u'pop_mobile1_abroad_pct', u'pop_5p_units_pct', u'pop_65p_pct',
       u'pop_disabled_pct', u'pop_school_priv_5_17_pct', u'pop_lim_eng_pct',
       u'unit_occ_rent_nochild_pct', u'unit_occ_hhsize_avg', u'pop_rent_pct',
       u'hh_u18_pct', u'pop_commute_time_avg', u'pop_commute_pubtrans_pct'],
      dtype='object')


,year,fc_geoid,total_vio_count,crime_prop_rt,crime_viol_rt,hh_alone_pct,pop_pov_pct,pop_edu_nohs_pct,unit_occ_rent_sevcrowd_pct,rent_gross_med,...,pop_65p_pct,pop_disabled_pct,pop_school_priv_5_17_pct,pop_lim_eng_pct,unit_occ_rent_nochild_pct,unit_occ_hhsize_avg,pop_rent_pct,hh_u18_pct,pop_commute_time_avg,pop_commute_pubtrans_pct
0,2014,36005000200,6.644144,7.977026,5.105297,0.145758,0.191997,0.248147,0.034662,1282.0,...,0.147399,0.111076,0.090508,0.278327,0.324090,3.69108,0.374092,0.426396,43.6244,0.340670
1,2014,36005000400,10.445026,5.789076,5.033979,0.151182,0.116187,0.147368,0.000000,1491.0,...,0.092606,0.095979,0.162991,0.137455,0.218935,3.05607,0.280510,0.409016,45.1009,0.462391
2,2014,36005001600,29.970588,9.706382,8.493084,0.303870,0.191159,0.329766,0.057254,920.0,...,0.160200,0.119555,0.085242,0.287729,0.391672,2.69195,0.689024,0.354603,42.9745,0.477379
3,2014,36005001900,4.661458,47.419805,22.315202,0.350120,0.445635,0.264921,0.176166,1126.0,...,0.036979,0.123794,0.032483,0.357023,0.183938,2.51199,0.756715,0.369305,46.1467,0.742391
4,2014,36005002000,14.404545,8.288640,13.164310,0.252349,0.421247,0.314322,0.031426,601.0,...,0.108494,0.214454,0.009434,0.219983,0.273167,3.00570,0.819288,0.476846,49.6604,0.673597


In [0]:
print(df_classC.columns)
df_classC.head()

Index([u'year', u'fc_geoid', u'class_C_count', u'crime_prop_rt',
       u'crime_viol_rt', u'hh_alone_pct', u'pop_pov_pct', u'pop_edu_nohs_pct',
       u'unit_occ_rent_sevcrowd_pct', u'rent_gross_med', u'hh_gini_idx',
       u'rent_burden_sev_pct', u'own_burden_pct', u'pop_foreign_pct',
       u'pop_mobile1_abroad_pct', u'pop_5p_units_pct', u'pop_65p_pct',
       u'pop_disabled_pct', u'pop_school_priv_5_17_pct', u'pop_lim_eng_pct',
       u'unit_occ_rent_nochild_pct', u'unit_occ_hhsize_avg', u'pop_rent_pct',
       u'hh_u18_pct', u'pop_commute_time_avg', u'pop_commute_pubtrans_pct'],
      dtype='object')


,year,fc_geoid,class_C_count,crime_prop_rt,crime_viol_rt,hh_alone_pct,pop_pov_pct,pop_edu_nohs_pct,unit_occ_rent_sevcrowd_pct,rent_gross_med,...,pop_65p_pct,pop_disabled_pct,pop_school_priv_5_17_pct,pop_lim_eng_pct,unit_occ_rent_nochild_pct,unit_occ_hhsize_avg,pop_rent_pct,hh_u18_pct,pop_commute_time_avg,pop_commute_pubtrans_pct
0,2014,36005000200,23.412698,7.977026,5.105297,0.145758,0.191997,0.248147,0.034662,1282.0,...,0.147399,0.111076,0.090508,0.278327,0.324090,3.69108,0.374092,0.426396,43.6244,0.340670
1,2014,36005000400,35.000000,5.789076,5.033979,0.151182,0.116187,0.147368,0.000000,1491.0,...,0.092606,0.095979,0.162991,0.137455,0.218935,3.05607,0.280510,0.409016,45.1009,0.462391
2,2014,36005001600,226.444444,9.706382,8.493084,0.303870,0.191159,0.329766,0.057254,920.0,...,0.160200,0.119555,0.085242,0.287729,0.391672,2.69195,0.689024,0.354603,42.9745,0.477379
3,2014,36005001900,16.272727,47.419805,22.315202,0.350120,0.445635,0.264921,0.176166,1126.0,...,0.036979,0.123794,0.032483,0.357023,0.183938,2.51199,0.756715,0.369305,46.1467,0.742391
4,2014,36005002000,77.292683,8.288640,13.164310,0.252349,0.421247,0.314322,0.031426,601.0,...,0.108494,0.214454,0.009434,0.219983,0.273167,3.00570,0.819288,0.476846,49.6604,0.673597


In [0]:
def qcut(data):
  for i in range(data.shape[1]):
    data.iloc[:,i] = pd.qcut(data.iloc[:,i].rank(method='first'), 5, labels=False)
  return data

for i in [2014,2015,2016,2017]:
  print('For {}:'.format(i))
  print
  total = qcut(df_total[df_total['year']==i].iloc[:,2:])
  classC = qcut(df_classC[df_classC['year']==i].iloc[:,2:])
  
  print('Total Count:')
  train,test=train_test_split(total,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'total_vio_count']==best_model.predict(train.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print "Out of sample:",(test.loc[:,'total_vio_count']==best_model.predict(test.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print 
  print('Class C:')
  train,test=train_test_split(classC,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'class_C_count']==best_model.predict(train.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print "Out of sample:",(test.loc[:,'class_C_count']==best_model.predict(test.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print
  print
  

For 2014:

Total Count:
Best Model:
[('unit_occ_hhsize_avg', 'hh_u18_pct'), ('pop_rent_pct', 'unit_occ_rent_sevcrowd_pct'), ('pop_rent_pct', 'pop_5p_units_pct'), ('pop_rent_pct', 'pop_commute_pubtrans_pct'), ('pop_5p_units_pct', 'hh_alone_pct'), ('pop_5p_units_pct', 'own_burden_pct'), ('pop_edu_nohs_pct', 'pop_pov_pct'), ('crime_viol_rt', 'crime_prop_rt'), ('crime_viol_rt', 'pop_school_priv_5_17_pct'), ('pop_lim_eng_pct', 'pop_edu_nohs_pct'), ('pop_foreign_pct', 'pop_lim_eng_pct'), ('pop_foreign_pct', 'unit_occ_rent_nochild_pct'), ('total_vio_count', 'crime_viol_rt'), ('rent_gross_med', 'pop_disabled_pct'), ('hh_alone_pct', 'unit_occ_hhsize_avg'), ('hh_u18_pct', 'rent_burden_sev_pct'), ('hh_u18_pct', 'pop_commute_time_avg'), ('hh_u18_pct', 'pop_65p_pct'), ('pop_pov_pct', 'hh_gini_idx'), ('pop_pov_pct', 'pop_rent_pct'), ('pop_pov_pct', 'total_vio_count'), ('pop_pov_pct', 'rent_gross_med')]
In sample: 0.44954128440366975
Out of sample: 0.37251908396946565

Class C:
Best Model:
[('unit_oc

Feature set 2

In [0]:
df['total_vio_count'] = df['unit_num']/df['total_vio_count']
df['class_C_count'] = df['unit_num']/df['class_C_count']
df=df.drop(columns=['unit_num'])
df_classC = df.drop(columns=['total_vio_count'])
df_total = df.drop(columns=['class_C_count'])

In [0]:
print(df_total.columns)
df_total.head()

Index([u'year', u'fc_geoid', u'total_vio_count', u'crime_prop_rt',
       u'crime_viol_rt', u'hh_family_pct', u'pop_race_div_idx', u'pop_pov_pct',
       u'pop_edu_collp_pct', u'unit_occ_rent_sevcrowd_pct', u'rent_gross_med',
       u'hh_gini_idx', u'pop_school_none_5_17_pct', u'rent_burden_sev_pct',
       u'own_burden_pct', u'pop_mobile1_abroad_pct',
       u'pop_commute_carfree_pct', u'pop_5p_units_pct', u'pop_65p_pct',
       u'pop_disabled_pct'],
      dtype='object')


,year,fc_geoid,total_vio_count,crime_prop_rt,crime_viol_rt,hh_family_pct,pop_race_div_idx,pop_pov_pct,pop_edu_collp_pct,unit_occ_rent_sevcrowd_pct,rent_gross_med,hh_gini_idx,pop_school_none_5_17_pct,rent_burden_sev_pct,own_burden_pct,pop_mobile1_abroad_pct,pop_commute_carfree_pct,pop_5p_units_pct,pop_65p_pct,pop_disabled_pct
0,2014,36005000200,6.644144,7.977026,5.105297,0.845540,0.480277,0.191997,0.146754,0.034662,1282.0,0.4024,0.069536,0.448669,0.529925,0.000198,0.361722,0.029469,0.147399,0.111076
1,2014,36005000400,10.445026,5.789076,5.033979,0.803738,0.503053,0.116187,0.283947,0.000000,1491.0,0.3659,0.024928,0.346512,0.507622,0.000000,0.486144,0.016550,0.092606,0.095979
2,2014,36005001600,29.970588,9.706382,8.493084,0.688285,0.445535,0.191159,0.144185,0.057254,920.0,0.4898,0.000000,0.246502,0.592000,0.012938,0.555902,0.583835,0.160200,0.119555
3,2014,36005001900,4.661458,47.419805,22.315202,0.561151,0.572817,0.445635,0.185140,0.176166,1126.0,0.4420,0.000000,0.319120,0.370968,0.001966,0.776087,0.614320,0.036979,0.123794
4,2014,36005002000,14.404545,8.288640,13.164310,0.737584,0.534598,0.421247,0.141399,0.031426,601.0,0.5266,0.020440,0.283613,0.580321,0.029834,0.711019,0.660489,0.108494,0.214454


In [0]:
print(df_classC.columns)
df_classC.head()

Index([u'year', u'fc_geoid', u'class_C_count', u'crime_prop_rt',
       u'crime_viol_rt', u'hh_family_pct', u'pop_race_div_idx', u'pop_pov_pct',
       u'pop_edu_collp_pct', u'unit_occ_rent_sevcrowd_pct', u'rent_gross_med',
       u'hh_gini_idx', u'pop_school_none_5_17_pct', u'rent_burden_sev_pct',
       u'own_burden_pct', u'pop_mobile1_abroad_pct',
       u'pop_commute_carfree_pct', u'pop_5p_units_pct', u'pop_65p_pct',
       u'pop_disabled_pct'],
      dtype='object')


,year,fc_geoid,class_C_count,crime_prop_rt,crime_viol_rt,hh_family_pct,pop_race_div_idx,pop_pov_pct,pop_edu_collp_pct,unit_occ_rent_sevcrowd_pct,rent_gross_med,hh_gini_idx,pop_school_none_5_17_pct,rent_burden_sev_pct,own_burden_pct,pop_mobile1_abroad_pct,pop_commute_carfree_pct,pop_5p_units_pct,pop_65p_pct,pop_disabled_pct
0,2014,36005000200,23.412698,7.977026,5.105297,0.845540,0.480277,0.191997,0.146754,0.034662,1282.0,0.4024,0.069536,0.448669,0.529925,0.000198,0.361722,0.029469,0.147399,0.111076
1,2014,36005000400,35.000000,5.789076,5.033979,0.803738,0.503053,0.116187,0.283947,0.000000,1491.0,0.3659,0.024928,0.346512,0.507622,0.000000,0.486144,0.016550,0.092606,0.095979
2,2014,36005001600,226.444444,9.706382,8.493084,0.688285,0.445535,0.191159,0.144185,0.057254,920.0,0.4898,0.000000,0.246502,0.592000,0.012938,0.555902,0.583835,0.160200,0.119555
3,2014,36005001900,16.272727,47.419805,22.315202,0.561151,0.572817,0.445635,0.185140,0.176166,1126.0,0.4420,0.000000,0.319120,0.370968,0.001966,0.776087,0.614320,0.036979,0.123794
4,2014,36005002000,77.292683,8.288640,13.164310,0.737584,0.534598,0.421247,0.141399,0.031426,601.0,0.5266,0.020440,0.283613,0.580321,0.029834,0.711019,0.660489,0.108494,0.214454


In [0]:
for i in [2014,2015,2016,2017]:
  print i
  print df[df['year']==i].shape
  

2014
(1636, 21)
2015
(1778, 21)
2016
(1795, 21)
2017
(1781, 21)


In [0]:
def qcut(data):
  for i in range(data.shape[1]):
    data.iloc[:,i] = pd.qcut(data.iloc[:,i].rank(method='first'), 5, labels=False)
  return data

for i in [2014,2015,2016,2017]:
  print('For {}:'.format(i))
  print
  total = qcut(df_total[df_total['year']==i].iloc[:,2:])
  classC = qcut(df_classC[df_classC['year']==i].iloc[:,2:])
  
  print('Total Count:')
  train,test=train_test_split(total,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'total_vio_count']==best_model.predict(train.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print "Out of sample:",(test.loc[:,'total_vio_count']==best_model.predict(test.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print 
  print('Class C:')
  train,test=train_test_split(classC,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'class_C_count']==best_model.predict(train.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print "Out of sample:",(test.loc[:,'class_C_count']==best_model.predict(test.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print
  print
  

For 2014:

Total Count:
Best Model:
[('pop_pov_pct', 'unit_occ_rent_sevcrowd_pct'), ('pop_pov_pct', 'hh_gini_idx'), ('pop_pov_pct', 'rent_burden_sev_pct'), ('pop_pov_pct', 'rent_gross_med'), ('pop_pov_pct', 'pop_edu_collp_pct'), ('pop_pov_pct', 'pop_commute_carfree_pct'), ('crime_viol_rt', 'crime_prop_rt'), ('pop_commute_carfree_pct', 'pop_5p_units_pct'), ('rent_gross_med', 'pop_disabled_pct'), ('pop_edu_collp_pct', 'own_burden_pct'), ('pop_5p_units_pct', 'hh_family_pct'), ('total_vio_count', 'pop_pov_pct'), ('total_vio_count', 'crime_viol_rt'), ('total_vio_count', 'pop_65p_pct')]
In sample: 0.44240570846075433
Out of sample: 0.3816793893129771

Class C:
Best Model:
[('pop_pov_pct', 'unit_occ_rent_sevcrowd_pct'), ('pop_pov_pct', 'hh_gini_idx'), ('pop_pov_pct', 'rent_burden_sev_pct'), ('pop_pov_pct', 'rent_gross_med'), ('pop_pov_pct', 'class_C_count'), ('crime_viol_rt', 'crime_prop_rt'), ('pop_commute_carfree_pct', 'pop_5p_units_pct'), ('rent_gross_med', 'pop_disabled_pct'), ('hh_family

without rescaling violent count

In [0]:
df=df.drop(columns=['unit_num'])
df_classC = df.drop(columns=['total_vio_count'])
df_total = df.drop(columns=['class_C_count'])

In [0]:
print(df_total.columns)
df_total.head()

Index([u'year', u'fc_geoid', u'total_vio_count', u'crime_prop_rt',
       u'crime_viol_rt', u'pop_num', u'pop_nonfam_pct', u'pop_race_div_idx',
       u'pop_pov_pct', u'pop_edu_collp_pct', u'unit_vac_pct',
       u'unit_occ_rent_sevcrowd_pct', u'rent_gross_med', u'hh_inc_med',
       u'hh_gini_idx'],
      dtype='object')


,year,fc_geoid,total_vio_count,crime_prop_rt,crime_viol_rt,pop_num,pop_nonfam_pct,pop_race_div_idx,pop_pov_pct,pop_edu_collp_pct,unit_vac_pct,unit_occ_rent_sevcrowd_pct,rent_gross_med,hh_inc_med,hh_gini_idx
0,2014,36005000200,222.0,7.977026,5.105297,5095,0.046712,0.480277,0.191997,0.146754,0.065085,0.034662,1282.0,74837.0,0.4024
1,2014,36005000400,191.0,5.789076,5.033979,5572,0.085248,0.503053,0.116187,0.283947,0.088221,0.000000,1491.0,77991.0,0.3659
2,2014,36005001600,68.0,9.706382,8.493084,5412,0.113821,0.445535,0.191159,0.144185,0.061825,0.057254,920.0,32354.0,0.4898
3,2014,36005001900,192.0,47.419805,22.315202,2569,0.181004,0.572817,0.445635,0.185140,0.068156,0.176166,1126.0,34635.0,0.4420
4,2014,36005002000,220.0,8.288640,13.164310,8959,0.095546,0.534598,0.421247,0.141399,0.059640,0.031426,601.0,23423.0,0.5266


In [0]:
BN(2014,'Total')

For 2014:

Total:
Best Model:
[('pop_pov_pct', 'hh_inc_med'), ('pop_pov_pct', 'hh_gini_idx'), ('crime_viol_rt', 'crime_prop_rt'), ('pop_edu_collp_pct', 'pop_nonfam_pct'), ('hh_inc_med', 'unit_occ_rent_sevcrowd_pct'), ('hh_inc_med', 'crime_viol_rt'), ('hh_inc_med', 'pop_edu_collp_pct'), ('hh_inc_med', 'rent_gross_med'), ('hh_inc_med', 'total_vio_count'), ('total_vio_count', 'pop_num')]
In sample: 0.41483516483516486
Out of sample: 0.401920438957476


In [0]:
df_total_14 = df_total[df_total['year']==2014].iloc[:,2:]
df_total_14.head()

,total_vio_count,crime_prop_rt,crime_viol_rt,pop_num,pop_nonfam_pct,pop_race_div_idx,pop_pov_pct,pop_edu_collp_pct,unit_vac_pct,unit_occ_rent_sevcrowd_pct,rent_gross_med,hh_inc_med,hh_gini_idx
0,6.644144,7.977026,5.105297,5095,0.046712,0.480277,0.191997,0.146754,0.065085,0.034662,1282.0,74837.0,0.4024
1,10.445026,5.789076,5.033979,5572,0.085248,0.503053,0.116187,0.283947,0.088221,0.000000,1491.0,77991.0,0.3659
2,29.970588,9.706382,8.493084,5412,0.113821,0.445535,0.191159,0.144185,0.061825,0.057254,920.0,32354.0,0.4898
3,4.661458,47.419805,22.315202,2569,0.181004,0.572817,0.445635,0.185140,0.068156,0.176166,1126.0,34635.0,0.4420
4,14.404545,8.288640,13.164310,8959,0.095546,0.534598,0.421247,0.141399,0.059640,0.031426,601.0,23423.0,0.5266


In [0]:
for i in range(df_total_14.shape[1]):
  df_total_14.iloc[:,i] = pd.qcut(df_total_14.iloc[:,i].rank(method='first'), 5, labels=False)

In [0]:
from sklearn.model_selection import train_test_split
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.estimators import BayesianEstimator


train,test=train_test_split(df_total_14,random_state=123,test_size=0.4)
hc = HillClimbSearch(train, scoring_method=BicScore(train))
best_model = hc.estimate()
print(best_model.edges())

[('pop_pov_pct', 'hh_inc_med'), ('pop_pov_pct', 'hh_gini_idx'), ('crime_viol_rt', 'crime_prop_rt'), ('pop_edu_collp_pct', 'pop_nonfam_pct'), ('hh_inc_med', 'unit_occ_rent_sevcrowd_pct'), ('hh_inc_med', 'crime_viol_rt'), ('hh_inc_med', 'pop_edu_collp_pct'), ('hh_inc_med', 'rent_gross_med'), ('hh_inc_med', 'total_vio_count'), ('total_vio_count', 'pop_num')]


drop median income

In [0]:
df['total_vio_count'] = df['unit_num']/df['total_vio_count']
df['class_C_count'] = df['unit_num']/df['class_C_count']
df=df.drop(columns=['unit_num','hh_inc_med'])
df_classC = df.drop(columns=['total_vio_count'])
df_total = df.drop(columns=['class_C_count'])

In [0]:
BN(2017,'Total')

For 2017:

Total:
Best Model:
[('pop_pov_pct', 'unit_occ_rent_sevcrowd_pct'), ('pop_pov_pct', 'hh_gini_idx'), ('pop_pov_pct', 'rent_gross_med'), ('pop_pov_pct', 'total_vio_count'), ('crime_viol_rt', 'crime_prop_rt'), ('rent_gross_med', 'pop_edu_collp_pct'), ('pop_edu_collp_pct', 'pop_nonfam_pct'), ('pop_nonfam_pct', 'unit_vac_pct'), ('total_vio_count', 'crime_viol_rt')]
In sample: 0.3951890034364261
Out of sample: 0.4066924066924067


violent count = number of unit / violent count

In [0]:
df['total_vio_count'] = df['unit_num']/df['total_vio_count']
df['class_C_count'] = df['unit_num']/df['class_C_count']
df=df.drop(columns=['unit_num'])
df_classC = df.drop(columns=['total_vio_count'])
df_total = df.drop(columns=['class_C_count'])

,year,fc_geoid,total_vio_count,crime_prop_rt,crime_viol_rt,pop_num,pop_nonfam_pct,pop_race_div_idx,pop_pov_pct,pop_edu_collp_pct,unit_vac_pct,unit_occ_rent_sevcrowd_pct,rent_gross_med,hh_inc_med,hh_gini_idx
0,2014,36005000200,6.644144,7.977026,5.105297,5095,0.046712,0.480277,0.191997,0.146754,0.065085,0.034662,1282.0,74837.0,0.4024
1,2014,36005000400,10.445026,5.789076,5.033979,5572,0.085248,0.503053,0.116187,0.283947,0.088221,0.000000,1491.0,77991.0,0.3659
2,2014,36005001600,29.970588,9.706382,8.493084,5412,0.113821,0.445535,0.191159,0.144185,0.061825,0.057254,920.0,32354.0,0.4898
3,2014,36005001900,4.661458,47.419805,22.315202,2569,0.181004,0.572817,0.445635,0.185140,0.068156,0.176166,1126.0,34635.0,0.4420
4,2014,36005002000,14.404545,8.288640,13.164310,8959,0.095546,0.534598,0.421247,0.141399,0.059640,0.031426,601.0,23423.0,0.5266


In [0]:
print(df_total.columns)
df_total.head()

Index([u'year', u'fc_geoid', u'total_vio_count', u'crime_prop_rt',
       u'crime_viol_rt', u'pop_num', u'pop_nonfam_pct', u'pop_race_div_idx',
       u'pop_pov_pct', u'pop_edu_collp_pct', u'unit_vac_pct',
       u'unit_occ_rent_sevcrowd_pct', u'rent_gross_med', u'hh_inc_med',
       u'hh_gini_idx'],
      dtype='object')


,year,fc_geoid,total_vio_count,crime_prop_rt,crime_viol_rt,pop_num,pop_nonfam_pct,pop_race_div_idx,pop_pov_pct,pop_edu_collp_pct,unit_vac_pct,unit_occ_rent_sevcrowd_pct,rent_gross_med,hh_inc_med,hh_gini_idx
0,2014,36005000200,6.644144,7.977026,5.105297,5095,0.046712,0.480277,0.191997,0.146754,0.065085,0.034662,1282.0,74837.0,0.4024
1,2014,36005000400,10.445026,5.789076,5.033979,5572,0.085248,0.503053,0.116187,0.283947,0.088221,0.000000,1491.0,77991.0,0.3659
2,2014,36005001600,29.970588,9.706382,8.493084,5412,0.113821,0.445535,0.191159,0.144185,0.061825,0.057254,920.0,32354.0,0.4898
3,2014,36005001900,4.661458,47.419805,22.315202,2569,0.181004,0.572817,0.445635,0.185140,0.068156,0.176166,1126.0,34635.0,0.4420
4,2014,36005002000,14.404545,8.288640,13.164310,8959,0.095546,0.534598,0.421247,0.141399,0.059640,0.031426,601.0,23423.0,0.5266


In [0]:
print(df_classC.columns)
df_classC.head()

Index([u'year', u'fc_geoid', u'class_C_count', u'crime_prop_rt',
       u'crime_viol_rt', u'pop_num', u'pop_nonfam_pct', u'pop_race_div_idx',
       u'pop_pov_pct', u'pop_edu_collp_pct', u'unit_vac_pct',
       u'unit_occ_rent_sevcrowd_pct', u'rent_gross_med', u'hh_inc_med',
       u'hh_gini_idx'],
      dtype='object')


,year,fc_geoid,class_C_count,crime_prop_rt,crime_viol_rt,pop_num,pop_nonfam_pct,pop_race_div_idx,pop_pov_pct,pop_edu_collp_pct,unit_vac_pct,unit_occ_rent_sevcrowd_pct,rent_gross_med,hh_inc_med,hh_gini_idx
0,2014,36005000200,23.412698,7.977026,5.105297,5095,0.046712,0.480277,0.191997,0.146754,0.065085,0.034662,1282.0,74837.0,0.4024
1,2014,36005000400,35.000000,5.789076,5.033979,5572,0.085248,0.503053,0.116187,0.283947,0.088221,0.000000,1491.0,77991.0,0.3659
2,2014,36005001600,226.444444,9.706382,8.493084,5412,0.113821,0.445535,0.191159,0.144185,0.061825,0.057254,920.0,32354.0,0.4898
3,2014,36005001900,16.272727,47.419805,22.315202,2569,0.181004,0.572817,0.445635,0.185140,0.068156,0.176166,1126.0,34635.0,0.4420
4,2014,36005002000,77.292683,8.288640,13.164310,8959,0.095546,0.534598,0.421247,0.141399,0.059640,0.031426,601.0,23423.0,0.5266


In [0]:
from sklearn.model_selection import train_test_split
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.estimators import BayesianEstimator

def qcut(data):
  for i in range(data.shape[1]):
    data.iloc[:,i] = pd.qcut(data.iloc[:,i].rank(method='first'), 5, labels=False)
  return data

for i in [2014,2015,2016,2017]:
  print('For {}:'.format(i))
  print
  total = qcut(df_total[df_total['year']==i].iloc[:,2:])
  classC = qcut(df_classC[df_classC['year']==i].iloc[:,2:])
  
  print('Total Count:')
  train,test=train_test_split(total,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'total_vio_count']==best_model.predict(train.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print "Out of sample:",(test.loc[:,'total_vio_count']==best_model.predict(test.drop(columns=['total_vio_count'])) \
                          ['total_vio_count']).mean()
  print 
  print('Class C:')
  train,test=train_test_split(classC,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  print "In sample:",(train.loc[:,'class_C_count']==best_model.predict(train.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print "Out of sample:",(test.loc[:,'class_C_count']==best_model.predict(test.drop(columns=['class_C_count'])) \
                          ['class_C_count']).mean()
  print
  print
  

 For 2014:

Total Count:
Best Model:
[('pop_pov_pct', 'hh_inc_med'), ('pop_pov_pct', 'hh_gini_idx'), ('crime_viol_rt', 'crime_prop_rt'), ('pop_edu_collp_pct', 'pop_nonfam_pct'), ('hh_inc_med', 'unit_occ_rent_sevcrowd_pct'), ('hh_inc_med', 'pop_edu_collp_pct'), ('hh_inc_med', 'rent_gross_med'), ('hh_inc_med', 'total_vio_count'), ('total_vio_count', 'crime_viol_rt')]
In sample: 0.3974358974358974
Out of sample: 0.40329218106995884

Class C:
Best Model:
[('pop_pov_pct', 'hh_inc_med'), ('pop_pov_pct', 'hh_gini_idx'), ('crime_viol_rt', 'crime_prop_rt'), ('crime_viol_rt', 'class_C_count'), ('pop_edu_collp_pct', 'pop_nonfam_pct'), ('hh_inc_med', 'unit_occ_rent_sevcrowd_pct'), ('hh_inc_med', 'crime_viol_rt'), ('hh_inc_med', 'pop_edu_collp_pct'), ('hh_inc_med', 'rent_gross_med')]
In sample: 0.3598901098901099
Out of sample: 0.3333333333333333


For 2015:

Total Count:
Best Model:
[('pop_pov_pct', 'unit_occ_rent_sevcrowd_pct'), ('crime_viol_rt', 'crime_prop_rt'), ('rent_gross_med', 'hh_inc_med')

In [0]:
from sklearn.model_selection import train_test_split
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.estimators import BayesianEstimator

def qcut(data):
  for i in range(data.shape[1]):
    data.iloc[:,i] = pd.qcut(data.iloc[:,i].rank(method='first'), 5, labels=False)
  return data


def BN(year,count):
  print('For {}:'.format(year))
  print
  if count=='Total':
    data_ = qcut(df_total[df_total['year']==year].iloc[:,2:])
  elif count=='Class C':
    data_ = qcut(df_classC[df_classC['year']==year].iloc[:,2:])
  
  print('{}:'.format(count))
  train,test=train_test_split(data_,random_state=123,test_size=0.4)
  hc = HillClimbSearch(train, scoring_method=BicScore(train))
  best_model = hc.estimate()
  print('Best Model:')
  print(best_model.edges())
  best_model.fit(train, estimator=BayesianEstimator, prior_type='K2')
  if count=='Total':
    print "In sample:",(train.loc[:,'total_vio_count']==best_model.predict(train.drop(columns=['total_vio_count'])) \
                            ['total_vio_count']).mean()
    print "Out of sample:",(test.loc[:,'total_vio_count']==best_model.predict(test.drop(columns=['total_vio_count'])) \
                            ['total_vio_count']).mean()
  elif count=='Class C':    
    print "In sample:",(train.loc[:,'class_C_count']==best_model.predict(train.drop(columns=['class_C_count'])) \
                            ['class_C_count']).mean()
    print "Out of sample:",(test.loc[:,'class_C_count']==best_model.predict(test.drop(columns=['class_C_count'])) \
                            ['class_C_count']).mean()


In [0]:
%%time
BN(2014,'Total')

For 2014:

Total:
Best Model:
[('pop_pov_pct', 'hh_inc_med'), ('pop_pov_pct', 'hh_gini_idx'), ('crime_viol_rt', 'crime_prop_rt'), ('pop_edu_collp_pct', 'pop_nonfam_pct'), ('hh_inc_med', 'unit_occ_rent_sevcrowd_pct'), ('hh_inc_med', 'pop_edu_collp_pct'), ('hh_inc_med', 'rent_gross_med'), ('hh_inc_med', 'total_vio_count'), ('total_vio_count', 'crime_viol_rt')]
In sample: 0.3974358974358974
Out of sample: 0.40329218106995884
CPU times: user 1min 5s, sys: 32 ms, total: 1min 5s
Wall time: 1min 5s
